In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
import bz2
import json
import datetime
from datetime import datetime as dt
from collections import defaultdict
from itertools import product
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm as pbar
from utils import tick_delta, dollars_to_ticks
import pickle as p

### TODO

* [x] Include time to market in final set
* [x] Include normalisation of the lay volume
* [x] Include the mid price in ticks
* Update code to include limits (i.e., only include 10 races)
* [x] Batch the saving of files into multiple files (Reduce the memory burden)
* [x] Include some simple MAs for linear models
* Update movement function on win type (batl moving to batb-5 for W, batb moving to batl+5)
* [x] Exclude the fucking index on the saving of files
* Move dataframe transformation into seperate file
* Resample to 1 second

Bot Updates

* The bot should trade only down
* The bot should enter a position according to optimisation over a loss function to set the threshold
* The bot should exit the position according to another exit threshold
* Implement the cashout function

In [4]:
run_id = dt.now().strftime('%m%d')
year = '2019'
month = 'Sep'
stem_path = '/Users/jake/Downloads/PRO 2/2019/Sep/'
gcloud_path = 'gs://h2o_temp/dev_sets/'
# n_limit = 10 # None for all files in set

In [5]:
pd.set_option('display.max_columns', 500)

In [7]:
def int_dict():
    return defaultdict(int)

def double_d():
    return defaultdict(dict)

In [8]:
def parse_markets(entries):
    market_defs = []
    market_defs_ids = []
    for entry in entries:
        for rc in entry['mc']:
            if 'marketDefinition' in rc.keys():
                market_defs.append(rc['marketDefinition'])
                market_defs_ids.append(rc['id'])
    
    mrkt_def = pd.DataFrame(market_defs)
    mrkt_def['market_id'] = market_defs_ids
    
    return mrkt_def

In [9]:
def parse_entries(entries):
    batl = []
    batb = []
    batl_v = {}
    batb_v = {}
    for n in range(6):
        batl_v[n] = []
        batb_v[n] = []
            
    mrkts = []
    times = []
    sids = []
    tot_vols = []
    in_play = []
    mean_trd = []
    mrkt_inplay = {}
    tvs = defaultdict(int_dict)
    back_ladder = defaultdict(double_d)
    lay_ladder = defaultdict(double_d)
    trd_ladder = defaultdict(int_dict)

    market_defs = []
    market_defs_ids = []
    
    for entry in entries: # should be map
            ts = entry['pt']
            change_sids = []
            change_mids = []
            for rc in entry['mc']:
                if 'marketDefinition' in rc.keys():
                    play_mid = rc['id']
                    mrkt_inplay[play_mid] = rc['marketDefinition']['inPlay']
                    market_defs. append(rc['marketDefinition'])
                    market_defs_ids.append(rc['id'])
                if 'rc' in rc.keys():
                    mid = rc['id']
                    for change in rc['rc']:
                        sid = change['id']
                        change_sids.append(sid)
                        change_mids.append(mid)
                        keys = change.keys()
                        if 'atb' in keys:
                            for prc, vol in change['atb']:
                                back_ladder[mid][sid][prc] = vol
                        if 'atl' in keys:
                            for prc, vol in change['atl']:
                                lay_ladder[mid][sid][prc] = vol
#                         if 'tv' in keys:
#                             tvs[mid][sid] = change['tv']
                        if 'trd' in keys:
                            for prc, vol in change['trd']:
                                trd_ladder[sid][prc] += vol
                        # TODO: Add in trade history: if 'trd' in keys:
            for sid, mid in zip(change_sids, change_mids):
                cur_batl = min([1000] + [k for k, v in lay_ladder[mid][sid].items() if v > 1])
                cur_batb = max([0] + [k for k, v in back_ladder[mid][sid].items() if v > 1])
#                         cur_tv = tvs[mid][sid]
                if cur_batl < 10 and cur_batb > 0:
                    tot_vol = 0
                    agg_vol = 0
                    for p, v in trd_ladder[sid].items():
                        agg_vol += float(p) * v
                        tot_vol += v
                    if tot_vol > 500:
                        mean_price = agg_vol / tot_vol if tot_vol > 1 else 0
                        mean_trd.append(mean_price)
                        times.append(ts)
                        sids.append(sid)
                        mrkts.append(mid)
                        batl.append(cur_batl)
                        batb.append(cur_batb)
                        for n in range(6):
                            adj_batl = tick_delta(cur_batl, n)
                            adj_batb = tick_delta(cur_batb, -n)
                            if adj_batl in lay_ladder[mid][sid].keys():
                                batl_v[n].append(lay_ladder[mid][sid][adj_batl])
                            else:
                                batl_v[n].append(None)
                            if adj_batb in back_ladder[mid][sid].keys():
                                batb_v[n].append(back_ladder[mid][sid][adj_batb])
                            else:
                                batb_v[n].append(None)
                        tot_vols.append(tot_vol)
                        in_play.append(mrkt_inplay[mid])

    df = pd.DataFrame({'timestamp': times, 
               'market_id': mrkts, 
               'sid': sids, 
               'batl': batl, 
               'batb': batb, 
               'batl_v': batl_v[0], 
               'batb_v': batb_v[0],
               'batl_v_1': batl_v[1], 
               'batb_v_1': batb_v[1], 
               'batl_v_2': batl_v[2], 
               'batb_v_2': batb_v[2], 
               'batl_v_3': batl_v[3], 
               'batb_v_3': batb_v[3], 
               'batl_v_4': batl_v[4], 
               'batb_v_4': batb_v[4], 
               'batl_v_5': batl_v[5], 
               'batb_v_5': batb_v[5], 
               'tot_vol': tot_vols,
                'mean_trd': mean_trd,
                'in_play': in_play})
    
#     df.timestamp = df.timestamp // 1000
#     df.timestamp = df.timestamp.apply(dt.fromtimestamp)
#     df.timestamp = pd.to_datetime(df.timestamp)
    
    mrkt_def = pd.DataFrame(market_defs)
    mrkt_def['market_id'] = market_defs_ids
#     keep_cols = ['market_id', 'eventId', 'eventTypeId', 'numberOfWinners','bettingType', 'marketType', 'marketTime', 'name', 'eventName']
#     mrkt_def = mrkt_def[keep_cols]
#     mrkt_def.drop_duplicates(inplace=True)
#     mrkt_def = mrkt_def[(mrkt_def.marketType=='WIN')]
#     mrkt_def.marketTime = pd.to_datetime(mrkt_def.marketTime)
#     mrkt_def.marketTime += np.timedelta64(10,'h')
    
    return df, mrkt_def

In [10]:
def find_pos(row):
    val = row.mid_ticks + 5
    idx = np.argwhere(idxs[row.sid] == row.inc_idx).flatten()[0]
    val = np.argmax(vals[row.sid][idx:] >= val)
    return val

def find_neg(row):
    val = row.mid_ticks - 5
    idx = np.argwhere(idxs[row.sid] == row.inc_idx).flatten()[0]
    val = np.argmax(vals[row.sid][idx:] <= val)
    return val

In [11]:
days = [x for x in os.listdir(stem_path) if len(x) <= 2]#[:2] # limit to first two days
final_paths = {}
for day in days:
    for fld in os.listdir(os.path.join(stem_path, day)):
        try:
            int(fld) # shit house validity check
            final_paths[fld] = os.path.join(stem_path, day, fld, fld+'.bz2')
        except:
            print(f'failed on {fld}')

In [12]:
count = len(final_paths.keys())
count

288

In [13]:
dir_path = f'../data/prepared/{run_id}_{year}_{month}'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

keep_cols = ['market_id', 'eventId', 'eventTypeId', 'numberOfWinners','bettingType', 'marketType', 'marketTime', 'name', 'eventName']

intermediate_cols = ['timestamp', 'test_starttime', 'market_id', 'sid', 'batl', 'batb', 'batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'tot_vol', 'mean_trd']

lay_vol_cols = ['batl_v', 'batl_v_1', 'batl_v_2', 'batl_v_3', 'batl_v_4', 'batl_v_5']

In [14]:
ma_cols = ['batl_ticks', 'batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'ticks_from_mean']
ma_periods = [5, 10, 50]

ma_features = []
for c, l in product(ma_cols, ma_periods):
    ma_features.append(f'{c}_ma_{l}')

In [16]:
feature_cols = ['batl_ticks', 'time_to_markt', 'batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'ticks_from_mean',
       'spread', 'ticks_1', 'ticks_2', 'ticks_3', 'ticks_4',
       'ticks_5', 'ticks_6', 'ticks_7', 'ticks_8', 'ticks_9', 'ticks_10',
       'ticks_11', 'ticks_12', 'ticks_13', 'ticks_14', 'ticks_15', 'ticks_16',
       'ticks_17', 'ticks_18', 'ticks_19', 'ticks_20', 'ticks_21', 'ticks_22',
       'ticks_23', 'ticks_24', 'ticks_25', 'ticks_26', 'ticks_27', 'ticks_28',
       'ticks_29', 'ticks_30', 'ticks_31', 'ticks_32', 'ticks_33', 'ticks_34',
       'ticks_35', 'ticks_36', 'ticks_37', 'ticks_38', 'ticks_39', 'ticks_40',
       'ticks_41', 'ticks_42', 'ticks_43', 'ticks_44', 'ticks_45', 'ticks_46',
       'ticks_47', 'ticks_48', 'ticks_49']

y_name = 'label'

feature_cols.extend(ma_features)

In [17]:
# TODO: Rename dfs thoughout to a single df and not get the overwrite on slice thing
dfs = []
defs = []
for k, v in pbar(final_paths.items(), '', count):
    try:
        fn = v

        entries = []
        with bz2.open(fn, mode='r') as f:
            for line in f.readlines():
                entries.append(json.loads(line))

        df, mrkt_def = parse_entries(entries)

        df = df.groupby(['timestamp','market_id', 'sid']).last().reset_index()
        df.timestamp = df.timestamp // 1000
        df.timestamp = df.timestamp.apply(dt.fromtimestamp)
        df.timestamp = pd.to_datetime(df.timestamp)

        mrkt_def = mrkt_def[keep_cols]
        mrkt_def.drop_duplicates(inplace=True)
        mrkt_def = mrkt_def[(mrkt_def.marketType=='WIN')]
        mrkt_def.marketTime = pd.to_datetime(mrkt_def.marketTime)
        mrkt_def.marketTime += np.timedelta64(10,'h')

        new_df = df.merge(mrkt_def, on='market_id', how='inner')
        new_df['test_starttime'] = new_df.marketTime.dt.tz_convert(None)
        new_df['test_recordtime'] = new_df.test_starttime - np.timedelta64(20,'m')
        new_df = new_df[(new_df.timestamp < new_df.test_starttime) & (new_df.timestamp > new_df.test_recordtime)]
        
        df = new_df

        df.fillna(0, inplace=True)

        df = df[intermediate_cols]

        df['time_to_markt'] = df.test_starttime - df.timestamp
        df['time_to_markt'] = df['time_to_markt'].astype(np.int) // 1000000000
        df['mid'] = (df.batb + df.batl) / 2

        for c in lay_vol_cols:
            df[c] = df[c] * df.mid

        price_cols = ['batl', 'batb', 'mean_trd', 'mid']
        for c in price_cols:
            df[f'{c}_ticks'] = df[c].apply(dollars_to_ticks)

        df['spread'] = df.batl_ticks - df.batb_ticks

        df['diff'] = df.groupby(['market_id', 'sid']).mid_ticks.diff().cumsum()

        df['trade_cnt'] = 1
        df['trade_cnt'] = df.groupby(['market_id', 'sid']).trade_cnt.cumsum()

        vals = {}
        idxs = {}
        for sid in df.sid.unique():
            vals[sid] = df[df.sid==sid].mid_ticks.values
            idxs[sid] = df[df.sid==sid].index.values

        df['inc_idx'] = df.index
        df['pos_idx'] = df.apply(find_pos, axis=1)
        df['neg_idx'] = df.apply(find_neg, axis=1)
        df['label'] = df.pos_idx < df.neg_idx

        for x in range(1, 50):
            df[f'ticks_{x}'] = df.mid_ticks - df.groupby(['market_id', 'sid']).mid_ticks.shift(x)

        df['ticks_from_mean'] = df.mid_ticks - df.mean_trd_ticks
        for c, l in product(ma_cols, ma_periods):
            df[f'{c}_ma_{l}'] = df[c].rolling(l).mean()

        df.dropna(inplace=True)
        df[feature_cols + [y_name]].to_csv(os.path.join(dir_path, f'{k}.csv'), index=False)
    except Exception as e:
        print(f'Failed on {k}, with exception: {e}')

/Users/jake/miniconda3/envs/betfair/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jake/miniconda3/envs/betfair/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jake/miniconda3/envs/betfair/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

Failed on 29475609, with exception: Cannot set a frame with no defined index and a value that cannot be converted to a Series
Failed on 29453654, with exception: Cannot set a frame with no defined index and a value that cannot be converted to a Series
Failed on 29425783, with exception: Cannot convert tz-naive timestamps, use tz_localize to localize
Failed on 29466021, with exception: Cannot set a frame with no defined index and a value that cannot be converted to a Series



In [18]:
os.environ['new_path'] = dir_path
os.environ['gcp_path'] = f'prepared/{run_id}_{year}_{month}'
!gsutil -m cp -r ./$new_path/*.csv gs://temp_h2o/$gcp_path

Copying file://./../data/prepared/0106_2019_Sep/29443900.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29443974.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29444059.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29444042.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29443999.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29444172.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29446120.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29447381.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29444091.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29448997.csv [Content-Type=text/csv]...
Copying file://./../data/prepared/0106_2019_Sep/29448964.csv [Content-Type=text/csv]...
Copying file://./../data/prepare

In [ ]:
df['time_to_markt'].astype(np.int) // 1000000000

In [ ]:
7*60 + 45

In [ ]:
df.tail()

In [ ]:
# dfs = []
# defs = []
# for k, v in pbar(final_paths.items(), '', count):
#     fn = v

#     entries = []
#     with bz2.open(fn, mode='r') as f:
#         for line in f.readlines():
#             entries.append(json.loads(line))

#     df, mrkt_def = parse_entries(entries)
    
    
#     dfs.append(df)
#     defs.append(mrkt_def)

In [ ]:
%time
final_df = pd.concat(dfs)

In [ ]:
final_df.shape

In [ ]:
%time
final_df = final_df.groupby(['timestamp','market_id', 'sid']).last().reset_index()

In [ ]:
%time
final_df.timestamp = final_df.timestamp // 1000
final_df.timestamp = final_df.timestamp.apply(dt.fromtimestamp)
final_df.timestamp = pd.to_datetime(final_df.timestamp)

In [ ]:
defs_df = pd.concat(defs)

In [ ]:

defs_df = defs_df[keep_cols]
defs_df.drop_duplicates(inplace=True)
defs_df = defs_df[(defs_df.marketType=='WIN')]
defs_df.marketTime = pd.to_datetime(defs_df.marketTime)
defs_df.marketTime += np.timedelta64(10,'h')

In [ ]:
new_df = final_df.merge(defs_df, on='market_id', how='inner')

In [ ]:
new_df['test_starttime'] = new_df.marketTime.dt.tz_convert(None)

In [ ]:
new_df['test_recordtime'] = new_df.test_starttime - np.timedelta64(20,'m')

In [ ]:
new_df = new_df[(new_df.timestamp < new_df.test_starttime) & (new_df.timestamp > new_df.test_recordtime)]

In [ ]:
new_df.shape

In [ ]:
new_df.duplicated().sum()

In [ ]:
new_df.to_csv(f'analysis_sets/{year}_{month}_trimmed.csv', index=False)

In [ ]:
df = new_df

In [ ]:
del new_df
del final_df
del defs_df
del dfs
del defs

In [ ]:
df.fillna(0, inplace=True)

keep_cols = ['timestamp', 'market_id', 'sid', 'batl', 'batb', 'batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'tot_vol', 'mean_trd']

df = df[keep_cols]

df['mid'] = (df.batb + df.batl) / 2

price_cols = ['batl', 'batb', 'mean_trd', 'mid']
for c in price_cols:
    df[f'{c}_ticks'] = df[c].apply(dollars_to_ticks)

df['spread'] = df.batl_ticks - df.batb_ticks

df['diff'] = df.groupby(['market_id', 'sid']).mid_ticks.diff().cumsum()

# temp = df[df.spread==1]
df['diff_ticks'] = df.groupby(['market_id', 'sid']).mid_ticks.diff()
df['diff_ticks'] = df.groupby(['market_id', 'sid']).diff_ticks.cumsum()

df['trade_cnt'] = 1
df['trade_cnt'] = df.groupby(['market_id', 'sid']).trade_cnt.cumsum()

vals = {}
idxs = {}
for sid in df.sid.unique():
    vals[sid] = df[df.sid==sid].mid_ticks.values
    idxs[sid] = df[df.sid==sid].index.values

def find_pos(row):
    val = row.mid_ticks + 5
    idx = np.argwhere(idxs[row.sid] == row.inc_idx).flatten()[0]
    val = np.argmax(vals[row.sid][idx:] >= val)
    return val

def find_neg(row):
    val = row.mid_ticks - 5
    idx = np.argwhere(idxs[row.sid] == row.inc_idx).flatten()[0]
    val = np.argmax(vals[row.sid][idx:] <= val)
    return val

df['inc_idx'] = df.index

df['pos_idx'] = df.apply(find_pos, axis=1)

df['neg_idx'] = df.apply(find_neg, axis=1)

df['label'] = df.pos_idx < df.neg_idx

for x in range(1, 50):
    df[f'ticks_{x}'] = df.mid_ticks - df.groupby(['market_id', 'sid']).mid_ticks.shift(x)

df['ticks_from_mean'] = df.mid_ticks - df.mean_trd_ticks

In [ ]:
feature_cols = ['batl_v', 'batb_v',
       'batl_v_1', 'batb_v_1', 'batl_v_2', 'batb_v_2', 'batl_v_3', 'batb_v_3',
       'batl_v_4', 'batb_v_4', 'batl_v_5', 'batb_v_5', 'ticks_from_mean',
       'spread', 'ticks_1', 'ticks_2', 'ticks_3', 'ticks_4',
       'ticks_5', 'ticks_6', 'ticks_7', 'ticks_8', 'ticks_9', 'ticks_10',
       'ticks_11', 'ticks_12', 'ticks_13', 'ticks_14', 'ticks_15', 'ticks_16',
       'ticks_17', 'ticks_18', 'ticks_19', 'ticks_20', 'ticks_21', 'ticks_22',
       'ticks_23', 'ticks_24', 'ticks_25', 'ticks_26', 'ticks_27', 'ticks_28',
       'ticks_29', 'ticks_30', 'ticks_31', 'ticks_32', 'ticks_33', 'ticks_34',
       'ticks_35', 'ticks_36', 'ticks_37', 'ticks_38', 'ticks_39', 'ticks_40',
       'ticks_41', 'ticks_42', 'ticks_43', 'ticks_44', 'ticks_45', 'ticks_46',
       'ticks_47', 'ticks_48', 'ticks_49']

y_name = 'label'

In [ ]:
pass_df = df[feature_cols + [y_name]].dropna()
pass_df.shape

In [ ]:
os.environ['FN'] = f'{year}_{month}.csv'
pass_df.to_csv('./test.csv')
!gsutil cp ./test.csv gs://temp_h2o/$FN